In [2]:
# CNN

from bs4 import BeautifulSoup
import requests

# Ask the user for the URL and modify it to start at page 1
url = input('Please put the URL ')
url = url + '?page=1'

# Ask the user for the tag name to be used in output filenames
tname = input('print the tag name')
a = 'Cnn_'
b = '_Wdate'
csv = '.csv'
simple = a + tname + csv
wd = a + tname + b + csv
stop_date = 20250231  # Set the date limit for articles

# Send a GET request to the first page to parse the layout
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
suggested_articles = soup.find('div', class_='flex-main')  # Locate main article container

searching_until = stop_date

# Initialize all data containers
date_list = []
tag_list = []
url_list = []
belonging_tag_list = []

# Function to update the URL for the next page
def new_url(local_url):
    global url
    u = url.find("=")
    curr_page_nr = local_url[u+1:]
    next_page_nr = str(int(curr_page_nr) + 1)
    url = local_url.replace(curr_page_nr, next_page_nr)

# Function to extract tags and date from an article
def tag_function(inside):
    global tag_list, date_list, searching_until
    inside_tag_list = []
    page = requests.get(inside)
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        date = soup.find("time").get('datetime')
        new_date = int("".join(date[:-15].split('-')))

        if new_date > searching_until:
            tag_section = soup.find('div', class_='item-tags')
            date_list.append(new_date)

            belonging_tag = soup.find('a', class_="main-item-category").get_text()

            for i in tag_section("a"):
                tag = i.get_text().replace("\n\t\t\t\t\t", "").replace("\t\t\t\t", "")
                inside_tag_list.append(tag)

            tag_list.append(inside_tag_list)

            new_btl = belonging_tag.replace('\n\t\t\t', "").replace("\t\t", "")
            belonging_tag_list.append(new_btl)

    except:
        print('Im in an article with propably no tags. \n PLEASE check the url: ', inside)

# List to track seen dates to detect duplicate pages
check_date_list = []

# Function to collect URLs of articles on the current page
def url_function():
    global url, tag_list, nr_of_art, searching_until, flag, check_date_list

    suggested_article_number = 0

    for h in suggested_articles('div', class_='gtr'):
        nr_of_art += 1
        a = h.find('a')
        urlone = a.get('href')
        https = "https://www.cnn.gr"

        # Prepend full domain if needed
        if urlone.find(https) < 0:
            urlone = https + urlone
        else:
            continue

        if urlone != 'None':
            suggested_article_number += 1

            if suggested_article_number == 1:
                # Extract date of first article to determine whether to continue
                page = requests.get(urlone)
                soup = BeautifulSoup(page.content, 'lxml')
                date = soup.find("time").get('datetime')
                new_date = int("".join(date[:-15].split('-')))

                # Check if already seen this date
                for i in check_date_list:
                    if new_date == i:
                        print('its the end of the new_url fun calling')
                        flag = False
                check_date_list.append(new_date)

                # If date is older than the stop date, exit loop
                if new_date < searching_until:
                    print('the date limit is here')
                    flag = False
                else:
                    url_list.append(urlone)
            else:
                url_list.append(urlone)

    # Normalize URLs again (defensive step)
    for i in range(len(url_list)):
        if url_list[i].find(https) < 0:
            url_list[i] = https + url_list[i]

# Main scraping loop
flag = True
nr_of_pages = 0
while flag:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    suggested_articles = soup.find('div', class_='flex-main')

    nr_of_art = 0
    url_function()

    if nr_of_art != 30:
        print('the nr of the aricles found is less than 30. So, the \n so, the articles are over')
        flag = False
    else:
        nr_of_pages += 1
        new_url(url)
        print(nr_of_pages)

print('I just got out of the while loop')

# Remove duplicate article URLs
url_list = list(set(url_list))

# Process each unique article to extract tags and dates
n = 0
for i in url_list:
    tag_function(i)
    print('im in the page ', n, " with the article url ", i)
    n += 1

import pandas as pd

# Flatten nested tag list into single list
new_tag_list = []
for i in range(len(tag_list)):
    for j in range(len(tag_list[i])):
        new_tag_list.append(tag_list[i][j])

# Count occurrences of each tag
res = {}
for i in new_tag_list:
    res[i] = new_tag_list.count(i)

# Convert tag count dictionary to list of pairs
resultList = new_list = list(map(list, res.items()))

# Save tag frequency to CSV
df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)

print('the programm is over! Enjoy the csv file!!')
print('the last date of the captured article is ', date_list[-1])
print('the length of the tag_list is ', len(tag_list))
print('the length of the date_list is ', len(date_list))

# Prepare dot-format dates for Tableau: dd.mm.yyyy
ddate_list = date_list
ttag_list = tag_list
ndlist = []
for i in ddate_list:
    i = str(i)
    y, m, d = i[:4], i[4:6], i[6:]
    ddate = d + '.' + m + '.' + y
    ndlist.append(ddate)

print(ddate_list)
print(ndlist)

# Build list of [tag, formatted_date] pairs
n = 0
date_con_list = []
for i in ttag_list:
    n += 1
    for tag in i:
        date_con_list.append([tag, ddate_list[n - 1]])

print(date_con_list)

# Save tag-date connections to CSV
df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)

# Final reporting
print(len(tag_list))
print(len(date_list))
print(tag_list)
print('the last date collected is ', date_list[-1])
print('the nr of the tags mentioned is ', len(resultList))


1
2
the date limit is here
3
I just got out of the while loop
im in the page  0  with the article url  https://www.cnn.gr/politiki/story/454387/dimoskopisi-metron-analysis-katalliloteros-o-mitsotakis-poios-einai-o-dimofilesteros-ypourgos
im in the page  1  with the article url  https://www.cnn.gr/politiki/story/478213/dimoskopisi-opinion-poll-eptakommatiki-vouli-sto-30-4-i-nd-paramenei-deyteri-i-pleysi
im in the page  2  with the article url  https://www.cnn.gr/politiki/story/448661/galazia-epixeirisi-asfyksias-tou-pasok-kai-perixarakosis-tou-mesaiou-xorou
im in the page  3  with the article url  https://www.cnn.gr/politiki/story/459034/dimoskopisi-interview-anodos-gia-ti-nea-dimokratia-ptosi-gia-pasok-ekto-komma-o-syriza
im in the page  4  with the article url  https://www.cnn.gr/politiki/story/459525/dimoskopisi-pulse-brosta-me-12-5-monades-i-nd-oi-protes-antidraseis-gia-ti-nea-epoxi-tramp
im in the page  5  with the article url  https://www.cnn.gr/politiki/video/471956/syreggela-sto